In [46]:
DOWNLOAD_DIR = "data"
!wget -nc -P $DOWNLOAD_DIR "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI HAR Dataset.zip"
!wget -nc -P $DOWNLOAD_DIR "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI HAR Dataset.names"
!unzip -oq -d $DOWNLOAD_DIR "$DOWNLOAD_DIR/UCI HAR Dataset.zip"

File 'data/UCI HAR Dataset.zip' already there; not retrieving.

File 'data/UCI HAR Dataset.names' already there; not retrieving.



In [47]:
import glob
import os
import numpy as np
from keras.utils.np_utils import to_categorical

DATA_DIR = os.path.join(DOWNLOAD_DIR, "UCI HAR Dataset")

X_train_list = [np.loadtxt(f) for f in glob.glob(os.path.join(DATA_DIR, "train", "Inertial Signals", "*"))]
X_test_list = [np.loadtxt(f) for f in glob.glob(os.path.join(DATA_DIR, "test", "Inertial Signals", "*"))]
X = np.r_['1', np.array(X_train_list), np.array(X_test_list)].transpose((1, 0, 2))

y_train = np.loadtxt(os.path.join(DATA_DIR, "train", "y_train.txt")) - 1
y_test = np.loadtxt(os.path.join(DATA_DIR, "test", "y_test.txt")) - 1
Y = to_categorical(np.r_[y_train, y_test])

train = np.arange(len(y_train))
test = np.arange(len(y_train), len(y_train) + len(y_test))

In [48]:
from keras.callbacks import Callback
from IPython.core.display import clear_output

class ClearOutput(Callback):
    def on_epoch_end(self, epoch, logs={}):
        clear_output()

In [49]:
from keras import backend as K
from keras.layers import Input, Dense, LSTM, Dropout
from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ProgbarLogger, TensorBoard

n_hidden = 32
lr = 0.0025
lambda_l2 = 0.0015
nb_epoch = 300
batch_size = 1500

input_shape = X_train.shape[1:]
output_shape = Y_train.shape[1]

model = Sequential()
model.add(LSTM(n_hidden, input_shape=input_shape, W_regularizer=l2(lambda_l2), return_sequences=True))
model.add(LSTM(n_hidden, W_regularizer=l2(lambda_l2)))
model.add(Dense(output_shape, activation='softmax'))
model.compile(optimizer=Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X[train], Y[train], nb_epoch=nb_epoch, batch_size=batch_size,
                    validation_data=(X[test], Y[test]),
                    callbacks=[ProgbarLogger(), TensorBoard(log_dir='log'), ClearOutput()])

7352/7352 [==============================] - 1s - loss: 0.1028 - acc: 0.9669 - val_loss: 0.4938 - val_acc: 0.8782


In [58]:
%%time
import cesium as cs

features_to_use = [f for f in cs.features.GENERAL_FEATS if 'period' not in f and 'flux' not in f]
fset_train = cs.featurize.featurize_time_series(None, [X_i for X_i in X[train]], None, 
                                                features_to_use=features_to_use,
                                                targets=y_train)

CPU times: user 54.8 s, sys: 3.82 s, total: 58.6 s
Wall time: 2min 24s


In [59]:
%%time
from sklearn.ensemble import RandomForestClassifier

#param_grid = {'n_estimators': [256, 512],
#              'criterion': ['gini', 'entropy'],
#              'max_features': [0.33, 0.5, 0.667]
#              }

cs_model = cs.build_model.build_model_from_featureset(fset_train, model_type='RandomForestClassifier',
#                                                      params_to_optimize=param_grid)
                                                      model_parameters={'n_estimators': 500})

CPU times: user 25.5 s, sys: 315 ms, total: 25.8 s
Wall time: 26.1 s


In [61]:
%%time
fset_test = cs.featurize.featurize_time_series(None, [X_i for X_i in X[test]], None, 
                                               features_to_use=features_to_use,
                                               targets=y_test)
cs_pred = cs.predict.model_predictions(fset_test, cs_model, return_probs=False)
print("Cesium test accuracy: {}".format(np.mean(cs_pred.prediction.values == y_test)))

Cesium test accuracy: 0.9178825924669155
CPU times: user 21.3 s, sys: 2.29 s, total: 23.6 s
Wall time: 54 s
